In [1]:
import sys
sys.path.append('..')
from iltransr.models.il_smiles_only import*
TD_param_file = os.path.join(root_path,'iltransr/pre-trained params/TD_best.params')

[03:18:47] ../src/storage/storage.cc:199: Using Pooled (Naive) StorageManager for CPU
[03:18:50] ../src/storage/storage.cc:199: Using Pooled (Naive) StorageManager for GPU


In [2]:
cnn_dropout = 0.1
batch_size = 64
bucket_num, bucket_ratio = 2, 0

In [ ]:
net = ILNet(dropout=cnn_dropout)
net.encoder = model.encoder
net.src_embed =  model.src_embed
net.hybridize()

In [4]:
net.load_parameters(TD_param_file,ctx=ctx)
net.initialize(init=mx.init.Xavier(magnitude=magnitude), ctx=ctx)

In [5]:
import pandas as pd
to_predict_data = pd.read_excel('smiles.xlsx')

In [6]:
predict_smiles = to_predict_data['SMILES'].map(canonical_smile).map(no_split).map(preprocess)
predict_smiles_lengths = to_predict_data['SMILES'].map(canonical_smile).map(no_split).map(get_length)
predict_sigma =mx.nd.ones((len(predict_smiles),1))
predict_dataset = gluon.data.SimpleDataset(gluon.data.ArrayDataset(predict_smiles,predict_sigma))
predict_dataloader = get_predict_dataloader(predict_dataset,predict_smiles_lengths,batch_size=batch_size, bucket_num=bucket_num,bucket_ratio=bucket_ratio)

In [7]:
predicted = predict(net, predict_dataloader,ctx)

[03:18:58] ../src/operator/cudnn_ops.cc:292: Auto-tuning cuDNN op, set MXNET_CUDNN_AUTOTUNE_DEFAULT to 0 to disable
[03:18:58] ../src/operator/cudnn_ops.cc:292: Auto-tuning cuDNN op, set MXNET_CUDNN_AUTOTUNE_DEFAULT to 0 to disable


In [8]:
df = pd.DataFrame(predicted)
df.to_csv('predicted_TD.csv')
print(df)

               0
0     271.333618
1     199.660629
2     217.839706
3     325.943207
4     203.042068
...          ...
5513  192.810394
5514  220.898209
5515  189.859451
5516  231.571686
5517  392.038849

[5518 rows x 1 columns]
